In [1]:
from termcolor import colored
import numpy as np
import cv2
from matplotlib.gridspec import GridSpec
from matplotlib import pyplot as plt
from scipy import ndimage, misc
from skimage.metrics import structural_similarity as ssim
from skimage import filters
import time
import progressbar
from pathlib import Path

files = [str(a) for a in Path("targetdir").glob('**/*.mp4')]

**Recalculate foundamental matrix from eight-point method using only background feature points**
this process can be reapplied iterativly until the foundamental matrix converge on a reliable result and moreover can be applied a RANSAC algorithm to clean from outliers

**Get epipolar lines from the two frame (feature points by SIFT)** 

In [3]:
def l1_distance(img1, img2):
    diff = np.abs(img1-img2)
    if img1.shape[-1] ==  3 and len(img1.shape)==3:
        diff = np.sum(diff, axis=-1)
    return diff

def l2_distance(img1, img2):
    sq_dist = (img1-img2)**2
    if img1.shape[-1] ==  3 and len(img1.shape)==3:
        sq_dist = np.sum(sq_dist,axis=-1)
    diff = np.sqrt(sq_dist)
    return diff

def max_distance(img1, img2):
    diff = np.abs(img1-img2)
    if img1.shape[-1] ==  3 and len(img1.shape)==3:
        diff = np.max(diff, axis=-1)
    return diff

In [4]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        # img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

In [5]:

#  https://www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/
# 

def auto_canny(image, sigma=0.33):
	''' lower value of sigma indicates a tighter threshold, whereas a larger value of sigma gives a wider threshold '''

	# compute the median of the single channel pixel intensities
	v = np.median(image)
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
	# return the edged image
	return edged

In [6]:
#  https://docs.opencv.org/4.5.1/d5/d0f/tutorial_py_gradients.html

def count_oriented(img, show=False):
  ''' count the points in a binary image taking in consideration the orientation '''

    # sobel derivatives
  derivX = cv2.Sobel(img, cv2.CV_32F, 1, 0)
  derivY = cv2.Sobel(img, cv2.CV_32F, 0, 1)

  mag = cv2.magnitude(np.absolute(derivX), np.absolute(derivY))
  
    # thresholding of the magnitude values
  thresh = 1000
  _, mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)
  mask = np.uint8(mask>0)
  
  if show:
    plt.figure(figsize=(15,10)); plt.title("Mask magnitude min: {} max: {}".format(np.int(np.min(mag[mag>0])), np.int(np.max(mag))))
    plt.imshow(cv2.cvtColor(mask*255, cv2.COLOR_BGR2RGB))
    plt.show()
  
  return np.sum(mask)

In [7]:
def expand_borders(m): return cv2.dilate(m, None, iterations=5)
def enhance_borders(im, k=np.array([[-1,-1,-1], [-1, 9,-1], [-1,-1,-1]])): return cv2.GaussianBlur(im, (3, 3), 0) # cv2.filter2D(im, -1, k) #

In [8]:
# This functions is used to determine if the bound (the next_bound) around the moving object is done correctly evaluating
# the difference between the two images in the ipotetical static zone:

index = 0

def spot_the_diff(image1_gray, image2_gray, log=False, show_work=True, debug_init=True):
    ''' Given two images return the one with the object  
          image1_gray: [prev_bound - next_bound] * prev_show
          image2_gray: [prev_bound - next_bound] * next_frame
    ret=> if bound was correct the diff function will find dirt in image1 (=>1) or image1 and image2 are equals (=>0)
          if bound was incorrect the diff function will contain some moving parts of image2 (=>2)
             (can happen that also identifies small part like light changes of image1 but returns 2) ''' 

    if debug_init:
        global index
        plt.figure(figsize=(15,15))
        plt.subplot(121); plt.title("PREV {}".format(index))
        plt.imshow(cv2.cvtColor(image1_gray, cv2.COLOR_BGR2RGB))
        plt.subplot(122); plt.title("NEXT")
        plt.imshow(cv2.cvtColor(image2_gray, cv2.COLOR_BGR2RGB))
        plt.show()
        plt.imsave("spot_debug/prev_{}.jpg".format(index),cv2.cvtColor(image1_gray, cv2.COLOR_BGR2RGB))
        plt.imsave("spot_debug/next_{}.jpg".format(index),cv2.cvtColor(image2_gray, cv2.COLOR_BGR2RGB))

    (score, sim) = ssim(image1_gray, image2_gray, full=True)
    sim = np.uint8(sim*255)
    thresh_sim =  np.uint8(np.logical_not(cv2.threshold(sim, 205, 255, cv2.THRESH_BINARY)[1])*255)
    thresh_sim = cv2.medianBlur(thresh_sim, 7)
    sum_sim = np.sum(thresh_sim>0)

      # lower bound (low thresh, high detection)
    if sum_sim<100:
        print(colored("Equality_l", 'blue'))
        return 0

    dif = cv2.absdiff(image1_gray, image2_gray)
    thresh_dif = cv2.medianBlur(dif, 5)

      # higher bound (high thresh, low detection)
    if not np.any(thresh_dif>50):
        print(colored("Equality_h", 'blue'))
        return 0

    if show_work:
        plt.figure(figsize=(20,20))
        plt.subplot(131); plt.title("ssim")
        plt.imshow(cv2.cvtColor(np.uint8(thresh_sim), cv2.COLOR_BGR2RGB))

    adaptive=0
    detected = 110
    a=None
    while(detected>80 if detected<100 else detected>105):
        a = np.uint8(thresh_dif>adaptive)*255
        detected = ((np.sum(a>0)/sum_sim)*100).round(3)
        if log: print("Not {}%"-format(detected.round(2)))
        adaptive+=10

    mask = cv2.morphologyEx(a,cv2.MORPH_CLOSE, np.ones((12,12),np.uint8))
    mask = cv2.medianBlur(mask,3)
    gradient = cv2.morphologyEx(mask, cv2.MORPH_GRADIENT, np.ones((6,6),np.uint8))

    if show_work:
        plt.subplot(132); plt.title("Mask closed [{}]".format(adaptive-10))
        plt.imshow(cv2.cvtColor(np.uint8(mask), cv2.COLOR_BGR2RGB))
        plt.subplot(133); plt.title("Mask gradient [{}%]".format(detected.round(1)))
        plt.imshow(cv2.cvtColor(np.uint8(gradient), cv2.COLOR_BGR2RGB))
        plt.show()

      # canny in the border of the movement areas
    blurred1 = enhance_borders(image1_gray*(gradient>0))
    blurred2 = enhance_borders(image2_gray*(gradient>0))

    edges1 = auto_canny(blurred1)
    edges2 = auto_canny(blurred2)

      # count of canny oriented pixels in the border of the movement areas 
    i1=count_oriented(edges1)
    i2=count_oriented(edges2)
    if log: print(i1)
    if log: print(i2)

    if show_work:
        plt.figure(figsize=(20,20))
        plt.subplot(121)
        plt.title("Canny 1 [{}]".format(i1))
        plt.imshow(cv2.cvtColor(edges1, cv2.COLOR_BGR2RGB))
        plt.subplot(122)
        plt.title("Canny 2 [{}]".format(i2))
        plt.imshow(cv2.cvtColor(edges2, cv2.COLOR_BGR2RGB))
        plt.show()

    if show_work:
        plt.figure(figsize=(20,20))
        plt.subplot(121); plt.title("index {} sim_score {}".format(index, (score*100).round(1), 15), fontsize=20); 
        plt.imshow(cv2.cvtColor(image1_gray*(mask>0), cv2.COLOR_BGR2RGB))
        plt.subplot(122)
        plt.imshow(cv2.cvtColor(image2_gray*(mask>0), cv2.COLOR_BGR2RGB))

      # prioritize the false negative cases to minimize the false positives 
    if i1>(i2+(0.01*i2)):
        if show_work: plt.title("Oggetto in 1", fontsize=20); plt.show()
        print (colored("#Oggetto in 1 [{}]".format(i1-i2),'blue')); return 1;
    else:
        if show_work: plt.title("Oggetto in 2", fontsize=20); plt.show()
        print(colored("#Oggetto in 2 [{}]".format(i2-i1),'blue')); return 2;

In [9]:
des_recovery = None

def warp (prev_show, prev_frame, frame, show_work=False, show_result=False):
      # find the keypoints and descriptors with SIFT
    global des_recovery
    global mean_prev

    if not des_recovery is None:
        kp1, des1 = des_recovery
    else:
        kp1, des1 = sift.detectAndCompute(prev_show,None)
        des_recovery = (kp1, des1)
    kp2, des2 = sift.detectAndCompute(frame,None)
    
      # determine if camera is static, 
    if len(kp1)!=0 and len(kp2)!=0:
      pp1 = np.array([[int(p.pt[0]),int(p.pt[1])] for p in kp1])
      pp2 = np.array([[int(p.pt[0]),int(p.pt[1])] for p in kp2])
      matches = int(np.sum((pp1[:,None] == pp2).all(2).any(1))/len(kp1)*100)
      if(matches<THRESH_STATIC_OR_MOVING): # the prev images need warping and old SIFT points are trashed
          print( colored("Warping", "cyan") )
          des_recovery = None
      else: # no need to warp
          return (prev_show, prev_frame, frame)

    matches1 = []; matches1_bg = []
    matches2 = []; matches2_bg = []
      # match the descriptors and build the corrspondences arrays of good matches
    matches_pairs = flann.knnMatch(des1, des2,k=2)
    for i,(m,n) in enumerate(matches_pairs):
        if m.distance < 0.7*n.distance:
            matches1.append(kp1[m.queryIdx].pt)
            matches2.append(kp2[m.trainIdx].pt)
    matches1 = np.asarray(matches1, np.int32)
    matches2 = np.asarray(matches2, np.int32)

    if len(matches1)>MIN_MATCH_COUNT:
        F, mask = cv2.findFundamentalMat(matches1,matches2,cv2.FM_LMEDS)
          # we select only inlier points
        matches1 = matches1[mask.ravel()==1]
        matches2 = matches2[mask.ravel()==1]

          # find epilines corresponding to points in left image (first image) and drawing its lines on right image
        lines2 = cv2.computeCorrespondEpilines(matches1.reshape(-1,1,2),1,F)
        lines2 = lines2.reshape(-1,3)

        if show_work:
            img3,img4 = drawlines(prev_show,frame,lines2,matches2,matches1)
            plt.figure(figsize=(20,10))
            plt.subplot(121); plt.title("lines {0} points {1}".format(len(lines2), len(matches1))); plt.imshow(img3)
            plt.subplot(122); plt.imshow(img4)
            plt.show()

          # keep only points near <1px to an epipolar line
        for pt1,pt2 in zip(matches1,matches2):
            pixels_err = PIXELS_ERR
            for line in lines2:
                pixels_err = min(pixels_err,abs(line[0] * pt1[0] + line[1] * pt1[1] + line[2]) / np.sqrt((line[0] * line[0]) + (line[1]*line[1])))
            if pixels_err<1:
                matches1_bg.append(pt1)
                matches2_bg.append(pt2)

        print("bg matches: ",len(matches1_bg),"/",len(matches1))

    if len(matches1_bg)>MIN_MATCH_COUNT:
          # retrieving perspective homography
        src = np.float32(matches1_bg)
        dst = np.float32(matches2_bg)
        H, masked = cv2.findHomography(src,dst, cv2.RANSAC, 5.0)
        dst_show = cv2.warpPerspective(np.uint16(prev_show)+1,H,(prev_frame.shape[1], prev_frame.shape[0]))
        dst = cv2.warpPerspective(np.uint16(prev_frame)+1,H,(prev_frame.shape[1], prev_frame.shape[0]))
        
          # correct the last pixel at margins
        dst_show[cv2.dilate(np.uint8(dst_show==0), None)>0]=0
        dst[cv2.dilate(np.uint8(dst==0), None)>0]=0

          # warp
        prev_show = np.uint8(((dst_show-1)*(dst_show>0))+((dst_show<1)*frame))
        prev_frame = np.uint8(((dst-1)*(dst>0))+((dst<1)*frame))
        mean_prev = [prev_show]

        if show_result:
            plt.figure(figsize=(20,10))
            plt.subplot(121); plt.title.set_text('Left warp')
            plt.imshow(cv2.cvtColor(dst,cv2.COLOR_GRAY2RGB))
            plt.subplot(122); plt.title.set_text('Compelete prev frame warped')
            plt.imshow(cv2.cvtColor(prev_frame,cv2.COLOR_GRAY2RGB))
            plt.show()
        
        return (prev_show, prev_frame, frame)
    else:
        print( colored("Not enough matches are found - {}/{}".format(len(matches1_bg), MIN_MATCH_COUNT), "red") )
        mean_prev = [prev_show]

        return None

**FLOW**

In [18]:
 # FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
sift = cv2.SIFT_create()
flann = cv2.FlannBasedMatcher(index_params,search_params)
 # WARP parameters
THRESH_STATIC_OR_MOVING = 12 # %
PIXELS_ERR = 1
MIN_MATCH_COUNT = 10
 #other parameters
sim_thresh = 165
min_rect_body_ratio_thresh = 0.005
min_rect_thresh = (5, 10)
frames_to_skip = 2
mean_prev_size = 0
show_work = False
debug_spot = False
show_result = False
 # iterators
index = 0
prev_bounds = []
frame = None
prev_frame = None
des_recovery = None
output_video = []
mean_prev = []
           

# video_name=files[np.random.randint(0,len(files))]
video_name="ex/test.avi"
cap = cv2.VideoCapture(video_name)
tot_frames = 0
while True:
    for _ in range(frames_to_skip): ret, frame_BGR = cap.read() # skip
    ret, frame_BGR = cap.read()
    if frame_BGR is None: break
    tot_frames+=1
cap.release()

cap = cv2.VideoCapture(video_name)
bar = progressbar.ProgressBar(maxval=tot_frames, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
# for i in range(100):
#     for _ in range(frames_to_skip): ret, frame_BGR = cap.read() # skip
#     ret, frame_BGR = cap.read()
while True:
    for _ in range(frames_to_skip): ret, frame_BGR = cap.read() # skip
    ret, frame_BGR = cap.read()
    if frame_BGR is None: break
    
    # restart
    start = time.time()
    index+=1
    prev_frame = frame
    frame = cv2.cvtColor(np.uint8(frame_BGR),cv2.COLOR_BGR2GRAY)
    if index<2:
        min_rect_body_ratio_thresh = (frame.shape[0] * frame.shape[1] ) * min_rect_body_ratio_thresh
        prev_show = frame.copy()
        mean_prev.append(prev_show.copy())
        continue
    handle_camera_movement = warp(prev_show, prev_frame, frame)
    if handle_camera_movement is None:
        frame = prev_frame
        continue

    prev_show, prev_frame, next_frame = handle_camera_movement

    # plt.figure(figsize=(15,10))
    # plt.subplot(131); plt.title("prev_show")
    # plt.imshow(cv2.cvtColor(np.uint8(cv2.GaussianBlur(next_frame, (1, 1), 0)), cv2.COLOR_GRAY2RGB))
    # plt.subplot(132); plt.title("next")
    # plt.imshow(cv2.cvtColor(np.uint8(cv2.GaussianBlur(next_frame, (3, 3), 0)), cv2.COLOR_GRAY2RGB))
    # plt.subplot(133); plt.title("next")
    # plt.imshow(cv2.cvtColor(np.uint8(cv2.GaussianBlur(next_frame, (5, 5), 0)), cv2.COLOR_GRAY2RGB))
    # plt.show()
    
      # identify movement areas in prev_frame to next_frame 
    (score, sim) = ssim(prev_frame, cv2.GaussianBlur(next_frame, (3, 3), 0), full=True)
    sim = np.uint8(sim*255)
    thresh = cv2.threshold(sim, sim_thresh, 1, cv2.THRESH_BINARY)[1]
    thresh = np.uint8(np.logical_not(thresh))
    thresh = cv2.medianBlur(thresh, 11) # remove small pieces
    thresh = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE, np.ones((9,9),np.uint8)) # unify pieces

    static_area = np.uint8(np.logical_not(thresh))
    movement_area = thresh

      # get rid of the frame margins
    if np.any(movement_area[0,10:-10]) and not np.any(movement_area[10,10:-10]):
        movement_area[:10,:] = 0
    if np.any(movement_area[-1,10:-10]) and not np.any(movement_area[-10,10:-10]):
        movement_area[-10:-1,:] = 0
    if np.any(movement_area[10:-10,0]) and not np.any(movement_area[10:-10,10]):
        movement_area[:,:10] = 0
    if np.any(movement_area[10:-10,-1]) and not np.any(movement_area[10:-10,-10]):
        movement_area[:,-10:-1] = 0
    
      # crete rectangles from countours
    rects = []
    bounds = []
    recovery_bounds = []
    contours, hierarchy = cv2.findContours(movement_area,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    bounded_image=movement_area*prev_frame
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        if w > min_rect_thresh[0] and h > min_rect_thresh[1]:
            m=np.zeros_like(movement_area, np.uint8)
            m[y:y+h,x:x+w] = 1
            rects.append({"x":x,"y":y,"w":w,"h":h,"m":m})
        
    print("rects: ", len(rects))

      # create indipendent bounds from rectangles
    while len(rects)!=0:
        x = rects[0]["x"];y = rects[0]["y"];w = rects[0]["w"];h = rects[0]["h"]
        m = rects[0]["m"]
        m_expanded = expand_borders(m) 
        cv2.rectangle(bounded_image,(x,y),(x+w,y+h),(255,255,0),2)
        rects.remove(rects[0])

        shape_modified = True
        while shape_modified:
            shape_modified = False
            for r_in in rects:
                if np.any(np.logical_and(m_expanded, r_in["m"])):
                    rects.remove(r_in)
                    cv2.rectangle(bounded_image,(r_in["x"],r_in["y"]),(r_in["x"]+r_in["w"],r_in["y"]+r_in["h"]),(115,115,0),1)
                    m[r_in["y"]:r_in["y"]+r_in["h"],r_in["x"]:r_in["x"]+r_in["w"]] = 1
                    shape_modified = True
                    break
        bounds.append(m)

    print("bounds: ", len(bounds))

      # get rid of the distorsion due to low movement or stopping object
    for prev_bound in prev_bounds:
        movement_zones = []
        for idx_bound in range(len(bounds)):
            if np.array_equal(np.logical_and(prev_bound, bounds[idx_bound]), prev_bound):
                movement_zones.append(idx_bound)
        if len(movement_zones)>=10: # (ignore it) push forward the previous bound to the next round
            for idx_zone in reversed(movement_zones):
                bounds.pop(idx_zone)
            recovery_bounds.append(prev_bound)
        elif len(movement_zones)>=2: # (use it) group and add the rectangle
            canvas = np.zeros(movement_area.shape)
            
            for idx_bound in reversed(movement_zones):
                canvas[bounds[idx_bound]>0] = 1
                bounds.pop(idx_bound)

            (y, x) = np.where(canvas == 1)
            (topy, topx) = (np.min(y), np.min(x))
            (bottomy, bottomx) = (np.max(y), np.max(x))
            cv2.rectangle(bounded_image,(topx,topy),(bottomx,bottomy),(115,115,0),3)
            canvas[topx:bottomx+1, topy:bottomy+1] = 1
            bounds.append(canvas)

      # improvement areas 
    a = [] # areas to ignore into zone [prev_bounds - next_bounds]. The remaining is updated with next_frame
    b = [] # areas to update into zone [next_bounds - prev_bounds]. Update with prev_frame
    recover_prev_bound = [0]*len(prev_bounds)
    if not(prev_bounds is None) or len(prev_bounds) != 0:
        for bound in bounds:
            a_current = []
            b_current = []
            recover_current_bound = []
            for prev_bound_id,prev_bound in enumerate(prev_bounds):
                if (np.sum(prev_bound[bound==1])/np.sum(bound))>0.5 or (np.sum(bound[prev_bound==1])/np.sum(prev_bound))>0.5:
                    print("--cycle-- bound matching")
                    if np.any(np.logical_and(prev_bound, bound)):
                        print("crossing")

                        static_prev_zone = np.uint8(np.logical_and(prev_bound, np.logical_not(bound)))
                        moving_next_zone = np.logical_and(np.logical_not(prev_bound), bound)
                        # if not np.all(prev_bound[bound==1]) and not np.all(bound[prev_bound==1]):
                        if np.any(static_prev_zone) and np.any(moving_next_zone):
                            imm = spot_the_diff(static_prev_zone*prev_show, static_prev_zone*next_frame, log=True, show_work=show_work, debug_init=debug_spot)
                            print("is moving")
                            if imm == 0 or imm == 1: # (update) images equality or obj residual in prev_show 
                                a_current.append(bound)
                                b.append(np.uint8(np.logical_not(moving_next_zone)))
                                recover_current_bound.append(bound)
                            else: # (ignore it) probably the bound wasn't recognized properly
                                area_to_ignore = np.uint8(np.logical_or(prev_bound,bound))
                                a_current.append(area_to_ignore)
                                recover_current_bound.append(area_to_ignore)
                        elif np.any(static_prev_zone):
                            imm = spot_the_diff(static_prev_zone*prev_show, static_prev_zone*next_frame, log=True, show_work=show_work, debug_init=debug_spot)
                            print("is getting smaller")
                            if imm == 0 or imm == 1: # (update) 
                                print("and there is good stuff")
                                a_current.append(bound)
                                recover_current_bound.append(bound)
                            else:
                                a_current.append(prev_bound) # (ignore it)
                                recover_current_bound.append(prev_bound)
                        else:
                            print("is getting bigger")
                            a_current.append(bound) # (ignore it)
                            recover_current_bound.append(bound)
            if len(a_current)>0:
                # plt.figure(figsize=(20,10))
                a_all = np.all(a_current, axis=0)
                a.append(a_all)
                # plt.subplot(131); plt.title("a")
                # plt.imshow(cv2.cvtColor(np.uint8(a_all*next_frame), cv2.COLOR_GRAY2RGB))
                if len(b_current)>0:
                    b_all = np.all(b_current, axis=0)
                    b.append(np.logical_and(np.logical_not(a_all), b_all))
                    # plt.subplot(132); plt.title("b")
                    # plt.imshow(cv2.cvtColor(np.uint8(b_all*prev_frame), cv2.COLOR_GRAY2RGB))
            if len(recover_current_bound)>0:
                rec_all = np.all(recover_current_bound, axis=0) 
                contours, hierarchy = cv2.findContours(np.uint8(rec_all),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                if len(contours)>1:
                    for idx in range(len(contours)):
                        canvas = np.zeros_like(next_frame, np.uint8)
                        cv2.drawContours(canvas, contours, idx, 1, -1)
                        recovery_bounds.append(canvas)
                else:
                    recovery_bounds.append(rec_all)
            #     plt.subplot(133); plt.title("rec {}".format(len(contours)))
            #     plt.imshow(cv2.cvtColor(np.uint8(rec_all)*255, cv2.COLOR_GRAY2RGB))
            # plt.show()
            
            # do we need to perform an additional check when the static_prev_area is divided in two and
            # (probably) spot the diff fail and return 1? under here is the code for that case 
            # if len(accros_bound)>0:
            #     # unify to not have duplicates
            #     canvas = np.zeros_like(next_frame, np.uint8)
            #     for idx in range(len(accros_bound)):
            #         cv2.drawContours(canvas, accros_bound, idx, 1, -1)
            #     contours, hierarchy = cv2.findContours(canvas,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            #     for idx in range(len(contours)):
            #         canvas = np.zeros_like(next_frame, np.uint8)
            #         cv2.drawContours(canvas, contours, idx, 1, -1)
            #         (y, x) = np.where(canvas == 1)
            #         (topy, topx) = (np.min(y), np.min(x))
            #         (bottomy, bottomx) = (np.max(y), np.max(x))
            #         if (np.sum(canvas)>min_rect_body_ratio_thresh) and (bottomx-topx) > min_rect_thresh[0] and (bottomy-topy) > min_rect_thresh[1] :
            #             bounds.append(canvas)
            # if recover_bound: recovery_bounds.append(bound)

    if show_result:
        plt.figure(figsize=(20,10))
        plt.subplot(131); plt.title("Static area estim")
        plt.imshow(cv2.cvtColor(np.uint8(static_area*next_frame), cv2.COLOR_GRAY2RGB))
        plt.subplot(132); plt.title("Movement area estim")
        plt.imshow(cv2.cvtColor(np.uint8(movement_area*prev_frame), cv2.COLOR_GRAY2RGB))

        # apply good static pixels
    for old_zone in b:
        prev_show[old_zone>0]=(old_zone*prev_show)[old_zone>0]
    for non_static_zone in a:
        static_area[non_static_zone>0] = 0
    if len(a)>0: prev_show[static_area>0]=(static_area*next_frame)[static_area>0]

    if show_result:
        plt.subplot(132); plt.title("Bounds")
        plt.imshow(cv2.cvtColor(np.uint8(bounded_image), cv2.COLOR_GRAY2RGB))
        plt.title("Show")
        plt.subplot(133); plt.imshow(cv2.cvtColor(np.uint8(prev_show), cv2.COLOR_GRAY2RGB))
        plt.show()
        
    mean_prev.append(prev_show.copy())
    if len(mean_prev)>mean_prev_size:
        mean_prev.pop(0)

    output_video.append(np.hstack([frame.copy(),np.mean(mean_prev, axis=0).copy() if mean_prev_size>0 else prev_show.copy()]))
    
    #log
    print(colored("bounds: {0} - prev_bounds: {1} - recovery_bounds: {2}".format(len(bounds),len(prev_bounds),len(recovery_bounds)),"yellow"))
    print(colored("time: {}".format(time.time()-start),"yellow"))
    print()
    
    #restart
    prev_bounds = recovery_bounds if len(prev_bounds) > 0 else bounds # ERR
    bar.update((index-1)+1)

cap.release()
bar.finish()
out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, (frame.shape[1]*2,frame.shape[0]))
 
for i in range(len(output_video)):
    out.write(cv2.cvtColor(np.uint8(output_video[i]), cv2.COLOR_GRAY2RGB))
    out.write(cv2.cvtColor(np.uint8(output_video[i]), cv2.COLOR_GRAY2RGB))
out.release()

# **########################## DEBUG ##############################**

In [0]:
spot_the_diff(good_zone*prev_show, good_zone*next_frame, True, True)

In [82]:
res1 = spot_the_diff(cv2.cvtColor(cv2.imread('spot_debug/prev_15.jpg'), cv2.COLOR_BGR2GRAY), cv2.cvtColor(cv2.imread('spot_debug/next_15.jpg'), cv2.COLOR_BGR2GRAY), True)
print(colored('no', 'red')) if(res1!=2) else print(colored('ok', 'green'))

Debug bound creation

<!-- plt.figure(figsize=(10,10))
plt.subplot(161)
plt.imshow(cv2.cvtColor(np.uint8((bounds[20]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(162)
plt.imshow(cv2.cvtColor(np.uint8((bounds[21]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(163)
plt.imshow(cv2.cvtColor(np.uint8((bounds[22]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(164)
plt.imshow(cv2.cvtColor(np.uint8((bounds[23]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(165)
plt.imshow(cv2.cvtColor(np.uint8((bounds[24]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(166)
plt.imshow(cv2.cvtColor(np.uint8((bounds[25]>0)*255), cv2.COLOR_GRAY2RGB))
plt.show()
plt.figure(figsize=(10,10))
plt.subplot(141)
plt.imshow(cv2.cvtColor(np.uint8((prev_bounds[0]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(142)
plt.imshow(cv2.cvtColor(np.uint8((prev_bounds[1]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(143)
plt.imshow(cv2.cvtColor(np.uint8((prev_bounds[2]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(144)
plt.imshow(cv2.cvtColor(np.uint8((prev_bounds[3]>0)*255), cv2.COLOR_GRAY2RGB))
plt.show()
plt.figure(figsize=(10,10))
plt.subplot(141)
plt.imshow(cv2.cvtColor(np.uint8((recovery_bounds[0]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(142)
plt.imshow(cv2.cvtColor(np.uint8((recovery_bounds[1]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(143)
plt.imshow(cv2.cvtColor(np.uint8((recovery_bounds[2]>0)*255), cv2.COLOR_GRAY2RGB))
plt.subplot(144)
plt.imshow(cv2.cvtColor(np.uint8((recovery_bounds[3]>0)*255), cv2.COLOR_GRAY2RGB))
plt.show() -->

### **########################################### TEST AND BACKUP ################################################**

Test `spot_the_diff`

<!-- res1 = spot_the_diff(cv2.cvtColor(cv2.imread('spot/1_2.png'), cv2.COLOR_BGR2GRAY), cv2.cvtColor(cv2.imread('spot/1.png'), cv2.COLOR_BGR2GRAY))
res4 = spot_the_diff(cv2.cvtColor(cv2.imread('spot/prev_low_thresh.jpg'), cv2.COLOR_BGR2GRAY), cv2.cvtColor(cv2.imread('spot/next_low_thresh.jpg'), cv2.COLOR_BGR2GRAY))
res5 = spot_the_diff(cv2.cvtColor(cv2.imread('spot/prev_high_thresh.jpg'), cv2.COLOR_BGR2GRAY), cv2.cvtColor(cv2.imread('spot/next_high_thresh.jpg'), cv2.COLOR_BGR2GRAY))
res6 = spot_the_diff(cv2.cvtColor(cv2.imread('spot_debug/prev_1.jpg'), cv2.COLOR_BGR2GRAY), cv2.cvtColor(cv2.imread('spot_debug/next_1.jpg'), cv2.COLOR_BGR2GRAY))
res7 = spot_the_diff(cv2.cvtColor(cv2.imread('spot_debug/next_2.jpg'), cv2.COLOR_BGR2GRAY), cv2.cvtColor(cv2.imread('spot_debug/prev_2.jpg'), cv2.COLOR_BGR2GRAY))
res8 = spot_the_diff(cv2.cvtColor(cv2.imread('spot_debug/next_3.jpg'), cv2.COLOR_BGR2GRAY), cv2.cvtColor(cv2.imread('spot_debug/prev_3.jpg'), cv2.COLOR_BGR2GRAY))
res9 = spot_the_diff(cv2.cvtColor(cv2.imread('spot_debug/next_4.jpg'), cv2.COLOR_BGR2GRAY), cv2.cvtColor(cv2.imread('spot_debug/prev_4.jpg'), cv2.COLOR_BGR2GRAY))
res10 = spot_the_diff(cv2.cvtColor(cv2.imread('spot_debug/next_5.jpg'), cv2.COLOR_BGR2GRAY), cv2.cvtColor(cv2.imread('spot_debug/prev_5.jpg'), cv2.COLOR_BGR2GRAY))
print(colored('no', 'red')) if(res1!=1) else print(colored('ok', 'green'))
print(colored('no', 'red')) if(res1!=2) else print(colored('ok', 'green'))
print(colored('no', 'red')) if(res4!=1) else print(colored('ok', 'green'))
print(colored('no', 'red')) if(res5!=1) else print(colored('ok', 'green'))
print(colored('no', 'red')) if(res6!=1) else print(colored('ok', 'green'))
print(colored('no', 'red')) if(res7!=1) else print(colored('ok', 'green'))
print(colored('no', 'red')) if(res8!=1) else print(colored('ok', 'green'))
print(colored('no', 'red')) if(res9!=1) else print(colored('ok', 'green'))
print(colored('no', 'red')) if(res10!=2) else print(colored('ok', 'green'))


# image1 = cv2.imread('12118.png')
# image2 = cv2.imread('12118_2.png')
# image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
# image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# image1_gray=good_zone*prev_show
# image2_gray=good_zone*next_frame
# spot_the_diff(good_zone*prev_show, good_zone*next_frame) -->


Backup `spot_the_diff`
<!-- index = 0

def spot_the_diff(image1_gray, image2_gray, log=False, show_work=True, debug_init=True):
    global index

    if debug_init:
        plt.figure(figsize=(20,20))
        plt.subplot(121); plt.title("Spot prev", fontsize=20)
        plt.imshow(cv2.cvtColor(image1_gray, cv2.COLOR_BGR2RGB))
        plt.subplot(122); plt.title("Spot next", fontsize=20)
        plt.imshow(cv2.cvtColor(image2_gray, cv2.COLOR_BGR2RGB))
        plt.show()

      # first i use a distance low
    dif = cv2.absdiff(image1_gray, image2_gray)
    a = cv2.threshold(dif, 50, 255, cv2.THRESH_BINARY)[1]
    a = cv2.medianBlur(a, 7)

    if not np.any(a):
        print(colored("Equality", 'blue'))
        return 0
    index = 0
    plt.imsave("spot_debug/prev_{}.jpg".format(index),cv2.cvtColor(image1_gray, cv2.COLOR_BGR2RGB))
    plt.imsave("spot_debug/next_{}.jpg".format(index),cv2.cvtColor(image2_gray, cv2.COLOR_BGR2RGB))
    detected = ((np.sum(a>0)/np.sum(image1_gray>0))*100).round(3)

    if(detected<30):
        if log: print("Not, ",detected)
        detected=0 
        adaptive=0
        thresh = max_distance(image1_gray, image2_gray)
        while(detected<25 or detected>65):
            a = np.uint8(thresh>adaptive)*255
            detected = ((np.sum(a>0)/np.sum(image1_gray>0))*100).round(3)
            if log: print("Not, ",detected)
            adaptive+=5
        a = cv2.erode(cv2.medianBlur(a, 21), np.ones((6,6),np.uint))
    else:
        adaptive=-1
        a = cv2.medianBlur(a, 15)

    detected = ((np.sum(a>0)/np.sum(image1_gray>0))*100).round(3)
    print(colored("#Detected count {0}% {1} ".format(detected, "absdiff" if adaptive<0 else adaptive-5),'blue'))
    if show_work:
        plt.figure(figsize=(20,20))
        plt.subplot(131); plt.title("Mask")
        plt.imshow(cv2.cvtColor(np.uint8(a), cv2.COLOR_BGR2RGB))

    # #contours,hierarchy = cv2.findContours(np.uint8(a), cv2.RETR_EXTERNAL, 1)
    # #p = np.zeros(image1_gray.shape)
    # #[cv2.drawContours(p, i, -1, (255),1) for i in contours]

    # #p = cv2.Canny(a,100,200)
    # #p = cv2.approxPolyDP(contours, 0.1*cv2.arcLength(contours,True), True)

    mask1 = cv2.morphologyEx(a,cv2.MORPH_CLOSE, np.ones((9,9),np.uint8))
    mask = cv2.morphologyEx(mask1, cv2.MORPH_GRADIENT, np.ones((6,6),np.uint8))
    if show_work:
        plt.subplot(132); plt.title("Mask closed")
        plt.imshow(cv2.cvtColor(np.uint8(mask1), cv2.COLOR_BGR2RGB))
        plt.subplot(133); plt.title("Mask gradient")
        plt.imshow(cv2.cvtColor(np.uint8(mask), cv2.COLOR_BGR2RGB))
        plt.show()

    # #mask = cv2.fillPoly(np.uint8(a), contours, 255) #-p
    # #mask = cv2.floodFill(np.uint8(a),np.uint8(a), 255, 5, 5, flags=cv2.FLOODFILL_MASK_ONLY) #-p

      # canny in the border of the movement areas
    edges = cv2.Canny(image1_gray*(mask>0),25,70)
    edges2 = cv2.Canny(image2_gray*(mask>0),30,90)

    if show_work:
        plt.figure(figsize=(20,20))
        plt.subplot(121)
        plt.title("canny 1")
        plt.imshow(cv2.cvtColor(edges, cv2.COLOR_BGR2RGB))
        plt.subplot(122)
        plt.title("canny 2")
        plt.imshow(cv2.cvtColor(edges2, cv2.COLOR_BGR2RGB))
        plt.show()

      # count of canny oriented pixels in the border of the movement areas 
    i1=count_oriented(edges)
    i2=count_oriented(edges2)
    if log: print(i1)
    if log: print(i2)

    if show_work:
        plt.figure(figsize=(20,20))
        plt.subplot(121)
        plt.imshow(cv2.cvtColor(image1_gray*(mask>0), cv2.COLOR_BGR2RGB))
        plt.subplot(122)
        plt.imshow(cv2.cvtColor(image2_gray*(mask>0), cv2.COLOR_BGR2RGB))

      # prioritize the false negative cases to minimize the false positives 
    if i1>(i2+(0.01*i2)):
        if show_work: plt.title("Oggetto in 1", fontsize=20); plt.show()
        print (colored("#Oggetto in 1",'blue')); return 1;
    else:
        if show_work: plt.title("Oggetto in 2", fontsize=20); plt.show()
        print(colored("#Oggetto in 2",'blue')); return 2; -->

Backup improvement areas
<!-- # improvement areas 
    a = [] # areas to ignore into zone [prev_bounds - next_bounds]. The remaining is updated with next_frame
    b = [] # areas to update into zone [next_bounds - prev_bounds]. Update with prev_frame
    recover_prev_bound = [0]*len(prev_bounds)
    if not(prev_bounds is None) or len(prev_bounds) != 0:
        for bound in bounds:
            accros_bound = []
            recover_bound = False
            for prev_bound_id,prev_bound in enumerate(prev_bounds):
                if (np.sum(prev_bound[bound==1])/np.sum(bound))>0.5 or (np.sum(bound[prev_bound==1])/np.sum(prev_bound))>0.5:
                    print("--cycle-- bound matching")
                    if np.any(np.logical_and(prev_bound, bound)):
                        print("crossing")

                        static_prev_zone = np.uint8(np.logical_and(prev_bound, np.logical_not(bound)))
                        # if not np.all(prev_bound[bound==1]) and not np.all(bound[prev_bound==1]):
                        if np.any(static_prev_zone) and np.any(np.logical_and(np.logical_not(prev_bound), bound)):
                            imm = spot_the_diff(static_prev_zone*prev_show, static_prev_zone*next_frame, log=True)
                            print("is moving")
                            if imm == 0 or imm == 1: # (update) images equality or obj residual in prev_show 
                                a.append(bound)
                                b.append(np.uint8(np.logical_not(static_prev_zone)))
                                contours, hierarchy = cv2.findContours(static_prev_zone,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                                if len(contours)>1: accros_bound += contours
                                else: recover_bound = True
                            else: # (ignore it) probably the bound wasn't recognized properly
                                area_to_ignore = np.uint8(np.logical_or(prev_bound,bound))
                                a.append(area_to_ignore)
                                recovery_bounds.append(area_to_ignore)
                                recover_prev_bound[prev_bound_id]=1
                        elif np.any(static_prev_zone):
                            imm = spot_the_diff(static_prev_zone*next_frame, static_prev_zone*prev_show, log=True)
                            print("is getting smaller")
                            if imm == 0 or imm == 1: # (update) 
                                print("and there is good stuff")
                                if recover_prev_bound[prev_bound_id] != 1:
                                    a.append(bound)
                                recover_bound = True
                            elif recover_prev_bound[prev_bound_id] != 1:
                                a.append(prev_bound) # (ignore it)
                                recovery_bounds.append(prev_bound)
                                recover_prev_bound[prev_bound_id]=1
                        else:
                            print("is getting bigger")
                            a.append(bound) # (ignore it)
                            recover_bound = True
            if len(accros_bound)>0:
                # unify to not have duplicates
                canvas = np.zeros_like(next_frame, np.uint8)
                for idx in range(len(accros_bound)):
                    cv2.drawContours(canvas, accros_bound, idx, 1, -1)
                contours, hierarchy = cv2.findContours(canvas,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                for idx in range(len(contours)):
                    canvas = np.zeros_like(next_frame, np.uint8)
                    cv2.drawContours(canvas, contours, idx, 1, -1)
                    (y, x) = np.where(canvas == 1)
                    (topy, topx) = (np.min(y), np.min(x))
                    (bottomy, bottomx) = (np.max(y), np.max(x))
                    if (np.sum(canvas)>min_rect_body_ratio_thresh) and (bottomx-topx) > min_rect_thresh[0] and (bottomy-topy) > min_rect_thresh[1] :
                        bounds.append(canvas)
            if recover_bound: recovery_bounds.append(bound) -->

Time test

<!-- start = time.time()
for _ in range(10000):
    a = np.logical_and(prev_bound, np.logical_not(bound));
    np.any(a) and np.any(np.logical_and(np.logical_not(prev_bound), bound)) 
print(time.time()- start)
start = time.time()
for _ in range(10000):
    a = np.logical_and(prev_bound, np.logical_not(bound));
    not np.all(prev_bound[bound==1]) and not np.all(bound[prev_bound==1])
print(time.time()- start)
start = time.time()
for _ in range(10000):
    a = np.logical_and(prev_bound, np.logical_not(bound));
    not np.all(prev_bound[bound==1]) and np.any(a) 
print(time.time()- start) -->

A matrix

<!-- a = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
])
a[5:6,8:10] = 1

np.uint8(np.logical_not(a)) -->

If old backup `code`
<!-- #--------------------------------------
#                     if not np.logical_and(np.all(np.any(accros, axis=0)[np.any(prev_bound, axis=0)]), np.all(np.any(accros, axis=1)[np.any(prev_bound, axis=1)])):
#                         div1 = accros.copy()
#                         div2 = accros.copy()
#                         if not np.all(np.any(accros, axis=0)[np.any(prev_bound, axis=0)]):
#                             col = np.argmin(prev_bound, axis=0)]) np.argmin(np.any(accros, axis=0)[np.any(prev_bound, axis=0)])
#                             div1[:,0:col] = 0
#                             div2[:,col:] = 0
#                         else:
#                             row = np.argmin(np.any(accros, axis=1)[np.any(prev_bound, axis=1)])
#                             div1[0:row,:] = 0
#                             div2[row:,:] = 0
#                         bounds.append(div1)
#                         bounds.append(div2)
#--------------------------------------
#                     if (((r_in["x"] >= x and r_in["x"] <= x+w) or (r_in["x"]+r_in["w"] >= x and r_in["x"]+r_in["w"] <= x+w))\
#                     and ((r_in["y"] >= y and r_in["y"] <= y+h) or (r_in["y"]+r_in["h"] >= y and r["y"]+r_in["h"] <= y+h))):
#--------------------------------------
#       min_y = np.int(np.argmax(canvas)/canvas.shape[1])
#       max_y = canvas.shape[0]-np.int(np.argmax(np.flip(canvas))/canvas.shape[1])
#       min_x = np.int(np.argmax(np.transpose(canvas))/canvas.shape[0])
#       max_x = canvas.shape[1]-np.int(np.argmax(np.transpose(np.flip(canvas)))/canvas.shape[0])
#-------------------------------------- -->

Frames to video `code`
<!-- out = cv2.VideoWriter('ex/lightonoff.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, (320, 240))
for i in range(1000,5001):
    name = 'image0{}.jpg'.format(i)
    imm = cv2.cvtColor(cv2.imread(name), cv2.COLOR_BGR2GRAY)
    out.write(cv2.cvtC
-->
Video for mean purpose
<!-- mean_prev=[]
output_video=[]
cap = cv2.VideoCapture("project.avi")
while True:
    ret, frame_BGR = cap.read()
    if frame_BGR is None: break
    frame = cv2.cvtColor(np.uint8(frame_BGR),cv2.COLOR_BGR2GRAY)
    real = frame[:frame.shape[0],:np.int(frame.shape[1]/2)]
    frame_mean = frame[:frame.shape[0],np.int(frame.shape[1]/2):]
    mean_prev.append(frame_mean.copy())
    if len(mean_prev)>5:
        mean_prev.pop(0)

    output_video.append(np.hstack([real.copy(),np.mean(mean_prev, axis=0).copy() if len(mean_prev)>1 else frame_mean.copy()]))
cap.release() -->

### **############################################### FAILS #######################################################**

**This is the wrong attempt counting the edges on the base of the orientation degree**

<!-- def count_oriented(img, show=False):
    # sobel derivatives
  derivX = cv2.Sobel(img, cv2.CV_32F, 1, 0)
  derivY = cv2.Sobel(img, cv2.CV_32F, 0, 1)
  
    # orientation and magnitude
  orienXY = cv2.phase(derivX, derivY, angleInDegrees=True)
  orienYX = cv2.phase(derivY, derivX, angleInDegrees=True)
  mag = cv2.magnitude(derivX, derivY)
  
    # thresholding of the magnitude values
  thresh = 50
  _, mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)
  
  if show:
      # set the colors
    red = np.asarray([0, 0, 255])
    cyan = np.asarray([255, 255, 0])
    green = np.asarray([0, 255, 0])
    yellow = np.asarray([0, 255, 255])

      # use np.uint8 required from OpenCV
    image_map = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

      # it checks that magnitude is above the threshold and that the orientation is in range
    image_map[ (mask == 255) & ((orienXY < 90) | (orienYX < 90))] = red
    # image_map[(mask == 255) & (((orienXY > 90) & (orienXY < 180)) | ((orienYX > 90) & (orienYX < 180)))] = cyan
    # image_map[(mask == 255) &  (((orienXY > 180) & (orienXY < 270)) | ((orienYX > 180) & (orienYX < 270)))] = green
    image_map[(mask == 255) & ((orienXY > 270) | (orienYX > 270))] = yellow

    plt.figure(figsize=(15,10)); plt.title("Mask magnitude min: {} max: {}".format(np.int(np.min(mag[mag>0])), np.int(np.max(mag))))
    plt.imshow(cv2.cvtColor(mask*255, cv2.COLOR_BGR2RGB))
    plt.show()
  
  return np.sum((mask == 255) & ((orienXY < 90) | (orienXY > 270) | (orienYX < 90) | (orienYX > 270))) -->

**This is the wrong attempt working on the moving shape - (PIC approach) Quantization of the pixels of the moving shape or of the full frame**

<!--
frameDelta = np.float32(cv2.absdiff(np.int32(prev_frame),np.int32(next_frame)))
thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]
thresh = cv2.dilate(cv2.medianBlur(thresh, 5), None, iterations=2)
bin_size = 5
shit = np.zeros(summed.shape)
for i in range(summed.shape[0]):
    for j in range(summed.shape[1]):
        if summed[i,j]:
            lst = [a["prev_pos"][i,j] for a in bins]
            histo, bin_seg = np.histogram(lst, bins=int(255/bin_size),range=[0,255])
            lb_bin = bin_seg[histo.argmax()]
            histo, bin_seg = np.histogram(histo, bins=bin_size-1,range=[lb_bin,lb_bin+bin_size-1])
            shit[i,j]=bin_seg[histo.argmax()]
plt.imshow(cv2.cvtColor(np.uint8(shit),cv2.COLOR_GRAY2RGB))
-->

**This is the wrong attempt working on the moving shape - (Mean and Median) interpolation of past frames through mean or median**

<!--
bg = []
idx = 0
# Initialize the background image
for at in range(10+num_frame_bg):
    ret, frame = cap.read()
    if ret and not frame is None:
        idx += 1
        frame = frame.astype(float)
        if idx > num_frame_bg:

            # Finding the background as the mean or median of first n frames
            wbg= bg*idx
            bg_interpolated = np.stack(wbg+[frame], axis=0)
            bg_interpolated = interpolation(bg_interpolated, axis=0)

            #plt.title("Background")
            #plt.imshow(cv2.cvtColor(bg_interpolated.astype(np.uint8), cv2.COLOR_BGR2RGB))
            #plt.show()

            ## Calculating a TFD
            #two_frame_difference(frame, bg_interpolated, distance, threshold)

            # Background subtraction
            mask = distance(frame, prev_frame) > threshold
            display_image_and_mask(bg_interpolated, mask)
            
            bg.pop(0)
            bg.append(bg_interpolated)
        else:
            bg.append(frame)
        prev_frame = frame
    else:
        break
#cap.release()
-->